In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
def ebay_search_by_title(query):
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://ebay.com")
    driver.maximize_window()

    # search bar
    search_bar = driver.find_element(By.ID, "gh-ac")
    search_bar.send_keys(query)
    search_bar.send_keys(Keys.ENTER)

    # sold button
    sold_checkbox = driver.find_element(By.CSS_SELECTOR, "input[aria-label='Sold Items']")
    sold_checkbox.click()

    # find image + titles + prices + URL
    images = driver.find_elements(By.XPATH, "//div[@class='s-item__image-wrapper image-treatment']/img")
    titles = driver.find_elements(By.CSS_SELECTOR, "span[role='heading']")
    prices = driver.find_elements(By.CSS_SELECTOR, "span[class='s-item__price']")
    urls = driver.find_elements(By.CSS_SELECTOR, "a[class='s-item__link']")

    # calculate price markers
    num_listings = len(titles)
    max_price = 0.0
    sum_prices = 0.0
    for i in range(1, num_listings):
        pattern = r'(\d+\.\d{2})'
        match = re.search(pattern, prices[i].text)
        price = float(match.group())
        max_price = max(max_price, price)
        sum_prices += price;
    avg_price = sum_prices / num_listings

    # format to two decimal places
    max_price = float("{:.2f}".format(max_price))
    avg_price = float("{:.2f}".format(avg_price))
    
    sold_listings_url = driver.current_url
    
    driver.quit()
    
    return (sold_listings_url, max_price, avg_price)


In [ ]:
import requests
import base64

url = "https://api.ebay.com/buy/browse/v1/item_summary/search_by_image?filter=buyingOptions:{FIXED_PRICE|BEST_OFFER}"

headers = {
    "Authorization": "Bearer v^1.1#i^1#f^0#p^1#I^3#r^0#t^H4sIAAAAAAAAAOVYXWwUVRTe7Q/YQMUgSoUmbAdQtJnZOzO729kpu3H7A93Yn6W7LLVR4e7M3TLt7sx0ZrbtKoXaB1DfAKMYHqgPmKgYEmsADVFphEQQK0JEBIwiUaIFjUF4MKHemZayrYS/bmIT92Uz55577vd995x7zwzomVbwxMaajVcL7dNz+npAT47dTs8ABdPyS+/PzZmXbwMZDva+nkU9eb25F5bqMJlQ+Uakq4qsI0dXMiHrvGX0ESlN5hWoSzovwyTSeUPgw4G6Wp6hAK9qiqEISoJwBKt8BFvmKmNjbtFd5vEgLw2xVb4eM6L4CMS6Ga+bi+MhFwNphMd1PYWCsm5A2fARDGBYkqZJwEUYmncDnvVSgAXNhCOKNF1SZOxCAcJvweWtuVoG1ltDhbqONAMHIfzBwLJwQyBYVV0fWerMiOUf1SFsQCOlj3+qVETkiMJECt16Gd3y5sMpQUC6Tjj9IyuMD8oHroO5B/iW1BzjcsdYlobuMhekGW9WpFymaElo3BqHaZFEMm658kg2JCN9O0WxGrFWJBijT/U4RLDKYf6tSMGEFJeQ5iOqKwJPB0Ihwt8I07USuayiDmptyCBDjVWYRYwDUPR4SI/AQIGh6dFVRkKNajxhmUpFFiVTMd1RrxgVCENGE4VxZQiDnRrkBi0QN0w4mX7sdQGZsmZzR0e2MGWslc1NRUmsgsN6vL38Y7MNQ5NiKQONRZg4YOnjI6CqSiIxcdBKxNHc6dJ9xFrDUHmns7Ozk+pkKUVrcTIA0M6mutqwsBYlcSVavmatY3/p9hNIyaIi4BrF/ryRVjGWLpyoGIDcQvhdnJtm3aO6j4fln2j9lyGDs3N8OWSrPBgXC0QQg5w77uU4KGajPPyjGeo0caAYTJNJK0PVBBQQKeA8SyWRJok8644zLBdHpOjxxkmXNx4n8aHoIek4QgChWEzwcv+bKrnTPA8jQUNGthI9O0keDlWU1jR11CTr61BrU7zhed3TFm5siapBro4FzkgkGurQKsM1zaUrfHdaCjclX5mQsDIRvH72BDBrPRsi1Ci6gcRJ0QsLiopCSkIS0lNrg1lNDEHNSIdRIoENkyIZUNVg1g7q7NC7mzPi3khn9Xb6L26mm7LSzXydWqzM+ToOAFWJMu8eSlCSTgXipsM0mbW+2kI9Kd4SblinFGtMcoStJI50mpRFmdI7BEpDupLScJNNNZi9V0RpQzK+zAxNSSSQFp1cXpvFnEymDBhLoKlW1VlIcAlOsZuW9ni9Lg43E95J8RKse3T1VDuSsnoO55XfRTftHP9i77dZP7rXPgB67R/n2O1gKVhMLwQl03JX5uXOnKdLBqIkGKd0qUXG76saotpQWoWSlvOg7Sr4ZbswVPP2K23XOtt/Lu+2ZX5X6HsWFI19WSjIpWdkfGYAxTdG8ulZcwsZlqYBx9BuwHqbwcIbo3n0w3lzqCO7vh3Y+Vvt5V89i3pP/OFjP23ZBArHnOz2fFter90WTe89Ejz21v6DzOwlVy59M7DwzScH1x2iLs8e7K73zDq59+zQpS2buOIHho8+13Fhdxj1DxbsudhRGj72wlO7jp0ZyH318aLvth7d204dHu5+d3juzn3nfhQXv3bq/P4r21ubrm5a2b/qhxNDa1ovpg+sW3Vm44z32xyuD5Z0e95bf/6Uq6jr5ZnLiwsD0a+3fFS+Xvrs0KPtjpbpB15kSHbB8jkv7vL2b35mw+Hzg4RjqOeh1Tu2dleWbJt3tro60bfBllrxyL5tb5ze8Xe7rXz+5+8sOjpI/PT7n1/sfGzzV9Vr9lypPdh4/OS1xX+9fl/JrPWfdH1/+dx8suha8viXpxd82DC82ys6+t0lkZeKfSN7+Q9Lgzov8REAAA==",
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Content-Language": "en-US"
}

In [ ]:
def ebay_search_by_image(encoded_image):
#     # Read the image file
#     with open("antique.jpg", "rb") as file:
#         image_data = file.read()

#     # Base64 encode the image data
#     encoded_image = base64.b64encode(image_data).decode("utf-8")

    request_data = {
        "image": encoded_image,
    }

    response = requests.post(url, headers=headers, json=request_data)
    data = response.json()

    max_price = 0.0
    sum_prices = 0.0
    num_items = 0.0
    for item in data['itemSummaries']:
        try:
            price = float(item['price']['value'])
            max_price = max(max_price, price)
            sum_prices += price
            num_items += 1.0
        except:
            continue

    return ("", round(sum_prices / num_items, 2), max_price)